![Logo BV IBMEC](https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/logo-bv-ibmec-notebooks.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ian-iania/IBMEC-BV-Modelos-Preditivos/blob/main/notebooks/NB01_Regression.ipynb)

# NB01 - Regressão e Séries Temporais (FP&A Banco BV)

**Objetivo:** aplicar regressão e séries temporais em dados sintéticos de FP&A do BV, com linguagem simples e foco em negócio.
**Vamos comparar baseline e modelos com split temporal e métricas MAE/RMSE, em fluxo linear.**
**A ideia é entender o processo sem exigir conhecimento prévio de programação.**


## 1) Setup do notebook

### O que vamos fazer
Importar as bibliotecas da aula.

### Por que isso importa para FP&A
Sem essas bibliotecas não conseguimos carregar dados, treinar modelos e visualizar resultados.

### O que você deve ver no output
Sem erro na execução.


In [ ]:
import pandas as pd  # biblioteca para carregar e manipular tabelas (DataFrames)
import numpy as np  # biblioteca para operações numéricas
import matplotlib.pyplot as plt  # biblioteca para criar gráficos
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet  # modelos lineares
from sklearn.tree import DecisionTreeRegressor  # modelo de árvore de decisão
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor  # modelos de ensemble baseados em árvores
from sklearn.metrics import mean_absolute_error, mean_squared_error  # métricas de avaliação MAE e RMSE
from statsmodels.tsa.holtwinters import ExponentialSmoothing  # modelo ETS (Holt-Winters)
from statsmodels.tsa.statespace.sarimax import SARIMAX  # modelo ARIMA/ARIMAX


## 2) PARTE A - Regressão com dataset linear (X -> y)

### O que vamos fazer
Carregar dataset linear, montar treino/teste e comparar baseline + modelos.

### Por que isso importa para FP&A
Esse tipo de modelo ajuda a explicar e prever a originação com base nos drivers.


In [ ]:
url_linear = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_regressao_linear.csv'  # URL do dataset linear
df_lin = pd.read_csv(url_linear)  # leitura do CSV para DataFrame
df_lin['ds'] = pd.to_datetime(df_lin['ds'])  # conversão da data para datetime
df_lin.head()  # exibe as primeiras linhas da base


### 2.1 Significado das colunas

- `ds`: data de referência mensal (início do mês)
- `y`: originação mensal em R$ milhões (**variável alvo**)
- `selic`: taxa SELIC
- `desemprego`: taxa de desemprego
- `ltv_medio`: LTV médio
- `spread`: spread médio
- `marketing`: índice sintético de esforço comercial
- `mix_auto`: participação de mix auto (0 a 1)
- `mes`: mês numérico (1 a 12)

Observação: `ds` e `y` são nomes comuns em modelagem (inclusive Prophet).

### 2.2 Inspeção da base (saídas separadas)

Agora vamos checar tamanho, colunas e tipos em células separadas para não misturar output.


In [ ]:
df_lin.shape  # mostra quantidade de linhas e colunas da base


In [ ]:
list(df_lin.columns)  # lista os nomes de todas as colunas


In [ ]:
df_lin.dtypes  # mostra o tipo de dado de cada coluna


### 2.3 Visualização inicial

### O que vamos fazer
Ver `y` ao longo do tempo e `selic` versus `y`.

### O que você deve ver no output
Um gráfico de linha e um de dispersão.


In [ ]:
plt.figure(figsize=(10, 4))  # cria a figura do gráfico temporal
plt.plot(df_lin['ds'], df_lin['y'], linewidth=2)  # plota y ao longo do tempo
plt.title('PARTE A - Originação (y) ao longo do tempo')  # define o título
plt.xlabel('Data')  # define eixo X
plt.ylabel('y (R$ milhões)')  # define eixo Y
plt.grid(alpha=0.3)  # adiciona grade
plt.show()  # renderiza gráfico


In [ ]:
plt.figure(figsize=(6, 4))  # cria a figura do gráfico de dispersão
plt.scatter(df_lin['selic'], df_lin['y'], alpha=0.75)  # plota selic no eixo X e y no eixo Y
plt.title('PARTE A - SELIC vs y')  # define título
plt.xlabel('SELIC')  # define eixo X
plt.ylabel('y (R$ milhões)')  # define eixo Y
plt.grid(alpha=0.3)  # adiciona grade
plt.show()  # renderiza gráfico


### 2.4 Split temporal e baseline

### O que vamos fazer
Separar treino/teste no tempo (80%/20%, sem embaralhar) e calcular baseline naive.

### Por que isso importa para FP&A
No mundo real, o modelo sempre prevê períodos futuros com base no passado.


In [ ]:
features = ['selic', 'desemprego', 'ltv_medio', 'spread', 'marketing', 'mix_auto', 'mes']  # lista de variáveis explicativas
split_idx = int(len(df_lin) * 0.8)  # define ponto de corte para treino e teste
train_lin = df_lin.iloc[:split_idx].copy()  # seleciona linhas de treino
train_lin = train_lin.sort_values('ds')  # garante ordenação temporal no treino
test_lin = df_lin.iloc[split_idx:].copy()  # seleciona linhas de teste
test_lin = test_lin.sort_values('ds')  # garante ordenação temporal no teste
X_train_lin = train_lin[features]  # cria matriz X de treino
X_test_lin = test_lin[features]  # cria matriz X de teste
y_train_lin = train_lin['y']  # cria vetor y de treino
y_test_lin = test_lin['y']  # cria vetor y de teste
print('Treino:', X_train_lin.shape, 'Teste:', X_test_lin.shape)  # imprime tamanhos de treino e teste
print('Período treino:', train_lin['ds'].min().date(), '->', train_lin['ds'].max().date())  # imprime intervalo de treino
print('Período teste :', test_lin['ds'].min().date(), '->', test_lin['ds'].max().date())  # imprime intervalo de teste


In [ ]:
baseline_value_lin = y_train_lin.iloc[-1]  # pega o último valor de y no treino
pred_baseline_lin = np.repeat(baseline_value_lin, len(y_test_lin))  # repete esse valor para todo o teste
mae_baseline_lin = mean_absolute_error(y_test_lin, pred_baseline_lin)  # calcula MAE do baseline
rmse_baseline_lin = np.sqrt(mean_squared_error(y_test_lin, pred_baseline_lin))  # calcula RMSE do baseline
print('Baseline MAE :', round(mae_baseline_lin, 3))  # imprime MAE
print('Baseline RMSE:', round(rmse_baseline_lin, 3))  # imprime RMSE


### 2.5 Modelos de regressão (execução linear)

Vamos treinar um modelo por vez, com uma célula de texto antes de cada treino.


#### Modelo 1: LinearRegression

**O que é:** regressão linear clássica.
**Por que usar:** ótimo ponto de partida e fácil de interpretar.


In [ ]:
model_lr = LinearRegression()  # cria o modelo de regressão linear
model_lr.fit(X_train_lin, y_train_lin)  # treina o modelo
pred_lr = model_lr.predict(X_test_lin)  # gera previsão no teste
mae_lr = mean_absolute_error(y_test_lin, pred_lr)  # calcula MAE
rmse_lr = np.sqrt(mean_squared_error(y_test_lin, pred_lr))  # calcula RMSE
print('LinearRegression RMSE:', round(rmse_lr, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** serve como referência de modelo supervisionado simples para comparar com modelos mais sofisticados.


#### Modelo 2: Ridge

**O que é:** regressão linear com regularização L2.
**Por que usar:** reduz sensibilidade a coeficientes extremos.


In [ ]:
model_ridge = Ridge(alpha=1.0, random_state=42)  # cria modelo Ridge
model_ridge.fit(X_train_lin, y_train_lin)  # treina o Ridge
pred_ridge = model_ridge.predict(X_test_lin)  # gera previsão no teste
mae_ridge = mean_absolute_error(y_test_lin, pred_ridge)  # calcula MAE
rmse_ridge = np.sqrt(mean_squared_error(y_test_lin, pred_ridge))  # calcula RMSE
print('Ridge RMSE:', round(rmse_ridge, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** mostra se uma versão regularizada melhora estabilidade em relação ao linear puro.


#### Modelo 3: ElasticNet

**O que é:** combina penalização L1 e L2.
**Por que usar:** pode ajudar quando há correlação entre variáveis.


In [ ]:
model_en = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000)  # cria modelo ElasticNet
model_en.fit(X_train_lin, y_train_lin)  # treina o ElasticNet
pred_en = model_en.predict(X_test_lin)  # gera previsão no teste
mae_en = mean_absolute_error(y_test_lin, pred_en)  # calcula MAE
rmse_en = np.sqrt(mean_squared_error(y_test_lin, pred_en))  # calcula RMSE
print('ElasticNet RMSE:', round(rmse_en, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** adiciona mais um candidato linear para comparação justa com modelos não-lineares.


#### Modelo 4: DecisionTree

**O que é:** árvore de decisão.
**Por que usar:** captura relações não-lineares de forma simples.


In [ ]:
model_tree = DecisionTreeRegressor(max_depth=4, random_state=42)  # cria modelo de árvore
model_tree.fit(X_train_lin, y_train_lin)  # treina árvore
pred_tree = model_tree.predict(X_test_lin)  # gera previsão no teste
mae_tree = mean_absolute_error(y_test_lin, pred_tree)  # calcula MAE
rmse_tree = np.sqrt(mean_squared_error(y_test_lin, pred_tree))  # calcula RMSE
print('DecisionTree RMSE:', round(rmse_tree, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** ajuda a verificar se padrões não-lineares já aparecem no dataset linear.


#### Modelo 5: RandomForest

**O que é:** conjunto de várias árvores (ensemble).
**Por que usar:** reduz variância da árvore única e costuma performar bem.


In [ ]:
model_rf = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42)  # cria RandomForest
model_rf.fit(X_train_lin, y_train_lin)  # treina RandomForest
pred_rf = model_rf.predict(X_test_lin)  # gera previsão no teste
mae_rf = mean_absolute_error(y_test_lin, pred_rf)  # calcula MAE
rmse_rf = np.sqrt(mean_squared_error(y_test_lin, pred_rf))  # calcula RMSE
print('RandomForest RMSE:', round(rmse_rf, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** é um modelo forte para comparação em dados tabulares de negócio.


#### Modelo 6: GradientBoosting

**O que é:** modelo de boosting com árvores sequenciais.
**Por que usar:** aprende correções progressivas e pode capturar padrões mais complexos.


In [ ]:
model_gbm = GradientBoostingRegressor(random_state=42)  # cria modelo GradientBoosting
model_gbm.fit(X_train_lin, y_train_lin)  # treina GradientBoosting
pred_gbm = model_gbm.predict(X_test_lin)  # gera previsão no teste
mae_gbm = mean_absolute_error(y_test_lin, pred_gbm)  # calcula MAE
rmse_gbm = np.sqrt(mean_squared_error(y_test_lin, pred_gbm))  # calcula RMSE
print('GradientBoosting RMSE:', round(rmse_gbm, 3))  # imprime RMSE do modelo


**Por que esse resultado importa:** permite testar um modelo não-linear poderoso em comparação com os demais.


### 2.6 Tabela final e gráfico comparativo (PARTE A)


In [ ]:
results_a = pd.DataFrame([  # cria tabela de métricas da PARTE A
    {'modelo': 'Baseline_Naive', 'mae': mae_baseline_lin, 'rmse': rmse_baseline_lin},  # baseline
    {'modelo': 'LinearRegression', 'mae': mae_lr, 'rmse': rmse_lr},  # linear
    {'modelo': 'Ridge', 'mae': mae_ridge, 'rmse': rmse_ridge},  # ridge
    {'modelo': 'ElasticNet', 'mae': mae_en, 'rmse': rmse_en},  # elasticnet
    {'modelo': 'DecisionTree', 'mae': mae_tree, 'rmse': rmse_tree},  # árvore
    {'modelo': 'RandomForest', 'mae': mae_rf, 'rmse': rmse_rf},  # random forest
    {'modelo': 'GradientBoosting', 'mae': mae_gbm, 'rmse': rmse_gbm},  # gradient boosting
]).sort_values('rmse').reset_index(drop=True)  # ordena por melhor RMSE
results_a  # exibe ranking final


In [ ]:
best_linear_name = results_a[results_a['modelo'].isin(['LinearRegression', 'Ridge', 'ElasticNet'])].iloc[0]['modelo']  # seleciona melhor linear
best_nonlinear_name = results_a[results_a['modelo'].isin(['DecisionTree', 'RandomForest', 'GradientBoosting'])].iloc[0]['modelo']  # seleciona melhor não-linear
pred_map_a = {  # dicionário com previsões da PARTE A
    'LinearRegression': pred_lr,  # previsões linear
    'Ridge': pred_ridge,  # previsões ridge
    'ElasticNet': pred_en,  # previsões elasticnet
    'DecisionTree': pred_tree,  # previsões árvore
    'RandomForest': pred_rf,  # previsões random forest
    'GradientBoosting': pred_gbm,  # previsões gradient boosting
}  # fim do dicionário
plt.figure(figsize=(11, 4))  # cria figura para comparação de previsões
plt.plot(test_lin['ds'], y_test_lin.values, label='Real', color='black', linewidth=2)  # linha real
plt.plot(test_lin['ds'], pred_baseline_lin, label='Baseline_Naive', linewidth=1.8)  # linha baseline
plt.plot(test_lin['ds'], pred_map_a[best_linear_name], label=f'Melhor linear: {best_linear_name}', linewidth=1.8)  # linha melhor linear
plt.plot(test_lin['ds'], pred_map_a[best_nonlinear_name], label=f'Melhor não-linear: {best_nonlinear_name}', linewidth=1.8)  # linha melhor não-linear
plt.title('PARTE A - Real vs Previsto (teste)')  # título do gráfico
plt.xlabel('Data')  # eixo X
plt.ylabel('y (R$ milhões)')  # eixo Y
plt.legend()  # legenda
plt.grid(alpha=0.3)  # grade
plt.show()  # renderiza gráfico


### 2.7 Feature importance (introdução)

### O que vamos fazer
Medir importância das variáveis para RandomForest e GradientBoosting.

### Por que isso importa para FP&A
Ajuda a priorizar quais drivers merecem atenção na discussão gerencial.

### Como interpretar
Importância alta sugere influência preditiva, mas **não prova causalidade**.


In [ ]:
rf_imp = pd.Series(model_rf.feature_importances_, index=features).sort_values(ascending=True)  # calcula importâncias do RandomForest
gbm_imp = pd.Series(model_gbm.feature_importances_, index=features).sort_values(ascending=True)  # calcula importâncias do GradientBoosting
rf_imp  # exibe vetor de importâncias do RandomForest


In [ ]:
plt.figure(figsize=(6, 4))  # cria figura do RandomForest
plt.barh(rf_imp.index, rf_imp.values)  # plota barras horizontais do RandomForest
plt.title('Importância - RandomForest')  # define título do gráfico
plt.grid(axis='x', alpha=0.3)  # adiciona grade horizontal
plt.show()  # renderiza gráfico


In [ ]:
plt.figure(figsize=(6, 4))  # cria figura do GradientBoosting
plt.barh(gbm_imp.index, gbm_imp.values)  # plota barras horizontais do GradientBoosting
plt.title('Importância - GradientBoosting')  # define título do gráfico
plt.grid(axis='x', alpha=0.3)  # adiciona grade horizontal
plt.show()  # renderiza gráfico


## 3) PARTE B - Mesmo pipeline no dataset não-linear

### O que vamos fazer
Repetir o fluxo no dataset não-linear.

### Por que isso importa para FP&A
Em cenários com limiares/interações, modelos não-lineares tendem a capturar melhor os padrões.


### 3.1 Carregar e preparar base não-linear


In [ ]:
url_nlin = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_regressao_nonlinear.csv'  # URL da base não-linear
df_nlin = pd.read_csv(url_nlin)  # leitura da base não-linear
df_nlin['ds'] = pd.to_datetime(df_nlin['ds'])  # conversão da data para datetime
df_nlin.head()  # exibe primeiras linhas


In [ ]:
split_idx_b = int(len(df_nlin) * 0.8)  # define ponto de corte de treino/teste
train_nlin = df_nlin.iloc[:split_idx_b].copy()  # recorte de treino da base não-linear
train_nlin = train_nlin.sort_values('ds')  # garante ordenação temporal de treino
test_nlin = df_nlin.iloc[split_idx_b:].copy()  # recorte de teste da base não-linear
test_nlin = test_nlin.sort_values('ds')  # garante ordenação temporal de teste
X_train_nlin = train_nlin[features]  # monta X de treino
X_test_nlin = test_nlin[features]  # monta X de teste
y_train_nlin = train_nlin['y']  # monta y de treino
y_test_nlin = test_nlin['y']  # monta y de teste
baseline_value_nlin = y_train_nlin.iloc[-1]  # captura último valor de treino para baseline
pred_baseline_nlin = np.repeat(baseline_value_nlin, len(y_test_nlin))  # gera previsão baseline
mae_baseline_nlin = mean_absolute_error(y_test_nlin, pred_baseline_nlin)  # calcula MAE do baseline
rmse_baseline_nlin = np.sqrt(mean_squared_error(y_test_nlin, pred_baseline_nlin))  # calcula RMSE do baseline
print('Baseline RMSE (não-linear):', round(rmse_baseline_nlin, 3))  # imprime RMSE baseline da PARTE B


### 3.2 Treinar modelos na base não-linear

Vamos repetir exatamente os mesmos modelos da PARTE A para comparar de forma justa.


#### Modelo LinearRegression (PARTE B)


In [ ]:
model_lr_b = LinearRegression()  # cria LinearRegression da PARTE B
model_lr_b.fit(X_train_nlin, y_train_nlin)  # treina LinearRegression na base não-linear
pred_lr_b = model_lr_b.predict(X_test_nlin)  # gera previsão da LinearRegression


#### Modelo Ridge (PARTE B)


In [ ]:
model_ridge_b = Ridge(alpha=1.0, random_state=42)  # cria Ridge da PARTE B
model_ridge_b.fit(X_train_nlin, y_train_nlin)  # treina Ridge na base não-linear
pred_ridge_b = model_ridge_b.predict(X_test_nlin)  # gera previsão do Ridge


#### Modelo ElasticNet (PARTE B)


In [ ]:
model_en_b = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000)  # cria ElasticNet da PARTE B
model_en_b.fit(X_train_nlin, y_train_nlin)  # treina ElasticNet na base não-linear
pred_en_b = model_en_b.predict(X_test_nlin)  # gera previsão do ElasticNet


#### Modelo DecisionTree (PARTE B)


In [ ]:
model_tree_b = DecisionTreeRegressor(max_depth=4, random_state=42)  # cria árvore da PARTE B
model_tree_b.fit(X_train_nlin, y_train_nlin)  # treina árvore na base não-linear
pred_tree_b = model_tree_b.predict(X_test_nlin)  # gera previsão da árvore


#### Modelo RandomForest (PARTE B)


In [ ]:
model_rf_b = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42)  # cria RandomForest da PARTE B
model_rf_b.fit(X_train_nlin, y_train_nlin)  # treina RandomForest na base não-linear
pred_rf_b = model_rf_b.predict(X_test_nlin)  # gera previsão do RandomForest


#### Modelo GradientBoosting (PARTE B)


In [ ]:
model_gbm_b = GradientBoostingRegressor(random_state=42)  # cria GradientBoosting da PARTE B
model_gbm_b.fit(X_train_nlin, y_train_nlin)  # treina GradientBoosting na base não-linear
pred_gbm_b = model_gbm_b.predict(X_test_nlin)  # gera previsão do GradientBoosting


### 3.3 Tabela de resultados da PARTE B


In [ ]:
# Garantia para evitar NameError se alguma célula de treino tiver sido pulada
if 'pred_lr_b' not in globals():  # verifica se previsões da PARTE B existem
    model_lr_b = LinearRegression()  # recria LinearRegression da PARTE B
    model_lr_b.fit(X_train_nlin, y_train_nlin)  # retreina LinearRegression da PARTE B
    pred_lr_b = model_lr_b.predict(X_test_nlin)  # regenera previsão da LinearRegression da PARTE B
if 'pred_ridge_b' not in globals():  # verifica previsão do Ridge na PARTE B
    model_ridge_b = Ridge(alpha=1.0, random_state=42)  # recria Ridge da PARTE B
    model_ridge_b.fit(X_train_nlin, y_train_nlin)  # retreina Ridge da PARTE B
    pred_ridge_b = model_ridge_b.predict(X_test_nlin)  # regenera previsão do Ridge da PARTE B
if 'pred_en_b' not in globals():  # verifica previsão do ElasticNet na PARTE B
    model_en_b = ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42, max_iter=5000)  # recria ElasticNet da PARTE B
    model_en_b.fit(X_train_nlin, y_train_nlin)  # retreina ElasticNet da PARTE B
    pred_en_b = model_en_b.predict(X_test_nlin)  # regenera previsão do ElasticNet da PARTE B
if 'pred_tree_b' not in globals():  # verifica previsão da árvore na PARTE B
    model_tree_b = DecisionTreeRegressor(max_depth=4, random_state=42)  # recria árvore da PARTE B
    model_tree_b.fit(X_train_nlin, y_train_nlin)  # retreina árvore da PARTE B
    pred_tree_b = model_tree_b.predict(X_test_nlin)  # regenera previsão da árvore da PARTE B
if 'pred_rf_b' not in globals():  # verifica previsão do RandomForest na PARTE B
    model_rf_b = RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42)  # recria RandomForest da PARTE B
    model_rf_b.fit(X_train_nlin, y_train_nlin)  # retreina RandomForest da PARTE B
    pred_rf_b = model_rf_b.predict(X_test_nlin)  # regenera previsão do RandomForest da PARTE B
if 'pred_gbm_b' not in globals():  # verifica previsão do GradientBoosting na PARTE B
    model_gbm_b = GradientBoostingRegressor(random_state=42)  # recria GradientBoosting da PARTE B
    model_gbm_b.fit(X_train_nlin, y_train_nlin)  # retreina GradientBoosting da PARTE B
    pred_gbm_b = model_gbm_b.predict(X_test_nlin)  # regenera previsão do GradientBoosting da PARTE B

results_b = pd.DataFrame([  # cria tabela de resultados da PARTE B
    {'modelo': 'Baseline_Naive', 'mae': mae_baseline_nlin, 'rmse': rmse_baseline_nlin},  # linha baseline
    {'modelo': 'LinearRegression', 'mae': mean_absolute_error(y_test_nlin, pred_lr_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_lr_b))},  # linha linear
    {'modelo': 'Ridge', 'mae': mean_absolute_error(y_test_nlin, pred_ridge_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_ridge_b))},  # linha ridge
    {'modelo': 'ElasticNet', 'mae': mean_absolute_error(y_test_nlin, pred_en_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_en_b))},  # linha elasticnet
    {'modelo': 'DecisionTree', 'mae': mean_absolute_error(y_test_nlin, pred_tree_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_tree_b))},  # linha árvore
    {'modelo': 'RandomForest', 'mae': mean_absolute_error(y_test_nlin, pred_rf_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_rf_b))},  # linha random forest
    {'modelo': 'GradientBoosting', 'mae': mean_absolute_error(y_test_nlin, pred_gbm_b), 'rmse': np.sqrt(mean_squared_error(y_test_nlin, pred_gbm_b))},  # linha gradient boosting
]).sort_values('rmse').reset_index(drop=True)  # ordena por melhor RMSE
results_b  # exibe ranking final da PARTE B


In [ ]:
best_linear_b = results_b[results_b['modelo'].isin(['LinearRegression', 'Ridge', 'ElasticNet'])].iloc[0]['modelo']  # identifica melhor linear da PARTE B
best_nonlinear_b = results_b[results_b['modelo'].isin(['DecisionTree', 'RandomForest', 'GradientBoosting'])].iloc[0]['modelo']  # identifica melhor não-linear da PARTE B
pred_map_b = {  # cria dicionário de previsões da PARTE B
    'LinearRegression': pred_lr_b,  # previsão LinearRegression
    'Ridge': pred_ridge_b,  # previsão Ridge
    'ElasticNet': pred_en_b,  # previsão ElasticNet
    'DecisionTree': pred_tree_b,  # previsão DecisionTree
    'RandomForest': pred_rf_b,  # previsão RandomForest
    'GradientBoosting': pred_gbm_b,  # previsão GradientBoosting
}  # fim do dicionário
plt.figure(figsize=(11, 4))  # cria figura comparativa da PARTE B
plt.plot(test_nlin['ds'], y_test_nlin.values, label='Real', color='black', linewidth=2)  # plota série real
plt.plot(test_nlin['ds'], pred_map_b[best_linear_b], label=f'Melhor linear: {best_linear_b}', linewidth=1.8)  # plota melhor linear
plt.plot(test_nlin['ds'], pred_map_b[best_nonlinear_b], label=f'Melhor não-linear: {best_nonlinear_b}', linewidth=1.8)  # plota melhor não-linear
plt.title('PARTE B - Melhor linear vs melhor não-linear (teste)')  # define título
plt.xlabel('Data')  # define eixo X
plt.ylabel('y (R$ milhões)')  # define eixo Y
plt.legend()  # exibe legenda
plt.grid(alpha=0.3)  # adiciona grade
plt.show()  # renderiza gráfico


## 4) PARTE C - Séries temporais (t -> y)

### O que vamos fazer
Comparar baseline, ETS, ARIMA, ARIMAX e Prophet (opcional).

### Por que isso importa para FP&A
Muitas previsões corporativas são feitas a partir do histórico da própria série.


In [ ]:
url_ts = 'https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_fpa_timeseries.csv'  # URL da base temporal
df_ts = pd.read_csv(url_ts)  # leitura da base temporal
df_ts['ds'] = pd.to_datetime(df_ts['ds'])  # conversão da data para datetime
df_ts.head()  # exibe primeiras linhas


In [ ]:
plt.figure(figsize=(10, 4))  # cria figura da série temporal
plt.plot(df_ts['ds'], df_ts['y'], linewidth=2)  # plota y ao longo do tempo
plt.title('PARTE C - Série temporal y')  # define título
plt.xlabel('Data')  # define eixo X
plt.ylabel('y (R$ milhões)')  # define eixo Y
plt.grid(alpha=0.3)  # adiciona grade
plt.show()  # renderiza gráfico


In [ ]:
holdout = 12  # define últimos 12 meses como teste
train_ts = df_ts.iloc[:-holdout].copy()  # separa treino temporal
test_ts = df_ts.iloc[-holdout:].copy()  # separa teste temporal
y_train_ts = train_ts['y'].values  # vetor y de treino
y_test_ts = test_ts['y'].values  # vetor y de teste
print('Treino:', train_ts.shape, 'Teste:', test_ts.shape)  # imprime tamanhos
print('Período treino:', train_ts['ds'].min().date(), '->', train_ts['ds'].max().date())  # imprime período de treino
print('Período teste :', test_ts['ds'].min().date(), '->', test_ts['ds'].max().date())  # imprime período de teste


In [ ]:
pred_naive_ts = np.repeat(y_train_ts[-1], len(y_test_ts))  # cria previsão naive do período de teste
mae_naive_ts = mean_absolute_error(y_test_ts, pred_naive_ts)  # calcula MAE baseline temporal
rmse_naive_ts = np.sqrt(mean_squared_error(y_test_ts, pred_naive_ts))  # calcula RMSE baseline temporal
print('Baseline MAE :', round(mae_naive_ts, 3))  # imprime MAE baseline temporal
print('Baseline RMSE:', round(rmse_naive_ts, 3))  # imprime RMSE baseline temporal


In [ ]:
ets = ExponentialSmoothing(train_ts['y'], trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True)  # ajusta ETS
aets = ets  # alias simples para leitura didática
pred_ets = aets.forecast(len(test_ts)).values  # prevê com ETS para período de teste
mae_ets = mean_absolute_error(y_test_ts, pred_ets)  # calcula MAE ETS
rmse_ets = np.sqrt(mean_squared_error(y_test_ts, pred_ets))  # calcula RMSE ETS
print('ETS MAE :', round(mae_ets, 3))  # imprime MAE ETS
print('ETS RMSE:', round(rmse_ets, 3))  # imprime RMSE ETS


In [ ]:
arima = SARIMAX(train_ts['y'], order=(1, 1, 1), enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)  # ajusta ARIMA
pred_arima = arima.forecast(steps=len(test_ts)).values  # prevê com ARIMA no período de teste
mae_arima = mean_absolute_error(y_test_ts, pred_arima)  # calcula MAE ARIMA
rmse_arima = np.sqrt(mean_squared_error(y_test_ts, pred_arima))  # calcula RMSE ARIMA
print('ARIMA MAE :', round(mae_arima, 3))  # imprime MAE ARIMA
print('ARIMA RMSE:', round(rmse_arima, 3))  # imprime RMSE ARIMA


In [ ]:
arimax = SARIMAX(train_ts['y'], exog=train_ts[['selic']], order=(1, 1, 1), enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)  # ajusta ARIMAX com selic
pred_arimax = arimax.forecast(steps=len(test_ts), exog=test_ts[['selic']]).values  # prevê com ARIMAX usando selic no teste
mae_arimax = mean_absolute_error(y_test_ts, pred_arimax)  # calcula MAE ARIMAX
rmse_arimax = np.sqrt(mean_squared_error(y_test_ts, pred_arimax))  # calcula RMSE ARIMAX
print('ARIMAX MAE :', round(mae_arimax, 3))  # imprime MAE ARIMAX
print('ARIMAX RMSE:', round(rmse_arimax, 3))  # imprime RMSE ARIMAX


### 4.1 Prophet (opcional)

Se não instalar, siga a aula normalmente com ETS/ARIMA/ARIMAX.


In [ ]:
import subprocess  # módulo para executar comando externo
import sys  # módulo para obter executável Python atual
prophet_ok = False  # flag de disponibilidade do Prophet
try:  # tenta importar Prophet já instalado
    from prophet import Prophet  # importa Prophet quando disponível
    prophet_ok = True  # marca disponibilidade
    print('Prophet já disponível.')  # mensagem de sucesso
except Exception:  # cai aqui quando Prophet não está instalado
    print('Tentando instalar Prophet...')  # informa tentativa de instalação
    try:  # tenta instalar via pip
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'prophet', '-q'])  # executa instalação
        from prophet import Prophet  # tenta importar novamente após instalação
        prophet_ok = True  # marca disponibilidade após instalar
        print('Prophet instalado com sucesso.')  # mensagem de sucesso
    except Exception as e:  # cai aqui quando instalação falha
        print('Prophet indisponível nesta sessão.')  # informa indisponibilidade
        print('Erro:', e)  # mostra detalhe do erro


In [ ]:
pred_prophet = None  # inicializa vetor de previsão Prophet
mae_prophet = None  # inicializa MAE Prophet
rmse_prophet = None  # inicializa RMSE Prophet
if prophet_ok:  # executa bloco somente se Prophet disponível
    train_prophet = train_ts[['ds', 'y', 'selic', 'evento']].copy()  # monta base de treino no formato Prophet
    test_prophet = test_ts[['ds', 'selic', 'evento']].copy()  # monta base de teste no formato Prophet
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)  # cria modelo Prophet
    m.add_regressor('selic')  # adiciona regressor selic
    m.add_regressor('evento')  # adiciona regressor evento
    m.fit(train_prophet)  # treina Prophet
    fcst = m.predict(test_prophet)  # prevê no período de teste
    pred_prophet = fcst['yhat'].values  # extrai vetor de previsão
    mae_prophet = mean_absolute_error(y_test_ts, pred_prophet)  # calcula MAE Prophet
    rmse_prophet = np.sqrt(mean_squared_error(y_test_ts, pred_prophet))  # calcula RMSE Prophet
    print('Prophet MAE :', round(mae_prophet, 3))  # imprime MAE Prophet
    print('Prophet RMSE:', round(rmse_prophet, 3))  # imprime RMSE Prophet
else:  # executa quando Prophet indisponível
    print('Prophet não executado.')  # informa que etapa foi pulada


In [ ]:
results_c = pd.DataFrame([  # cria tabela inicial da PARTE C
    {'modelo': 'Baseline_Naive', 'mae': mae_naive_ts, 'rmse': rmse_naive_ts},  # linha baseline
    {'modelo': 'ETS_HoltWinters', 'mae': mae_ets, 'rmse': rmse_ets},  # linha ETS
    {'modelo': 'ARIMA', 'mae': mae_arima, 'rmse': rmse_arima},  # linha ARIMA
    {'modelo': 'ARIMAX_selic', 'mae': mae_arimax, 'rmse': rmse_arimax},  # linha ARIMAX
])  # fecha DataFrame inicial
if pred_prophet is not None:  # verifica se Prophet está disponível
    results_c = pd.concat([results_c, pd.DataFrame([{'modelo': 'Prophet', 'mae': mae_prophet, 'rmse': rmse_prophet}])], ignore_index=True)  # acrescenta linha Prophet
results_c = results_c.sort_values('rmse').reset_index(drop=True)  # ordena por melhor RMSE
results_c  # exibe tabela final da PARTE C


In [ ]:
plt.figure(figsize=(11, 4))  # cria figura do gráfico final da PARTE C
plt.plot(test_ts['ds'], y_test_ts, label='Real', color='black', linewidth=2)  # plota série real
plt.plot(test_ts['ds'], pred_naive_ts, label='Baseline_Naive', linewidth=1.8)  # plota baseline
plt.plot(test_ts['ds'], pred_ets, label='ETS_HoltWinters', linewidth=1.8)  # plota ETS
plt.plot(test_ts['ds'], pred_arimax, label='ARIMAX_selic', linewidth=1.8)  # plota ARIMAX
if pred_prophet is not None:  # verifica se previsão Prophet existe
    plt.plot(test_ts['ds'], pred_prophet, label='Prophet', linewidth=1.8)  # plota Prophet quando disponível
plt.title('PARTE C - Real vs Previsões (teste)')  # define título
plt.xlabel('Data')  # define eixo X
plt.ylabel('y (R$ milhões)')  # define eixo Y
plt.legend()  # mostra legenda
plt.grid(alpha=0.3)  # adiciona grade
plt.show()  # renderiza gráfico


## 5) Conclusão didática

- **X -> y (regressão):** útil quando temos drivers para explicar a originação.
- **t -> y (série temporal):** útil quando o histórico da própria série é forte.
- **Regra prática:** validar no tempo e sempre comparar com baseline.


## 6) Checklist FP&A

- Baseline: compare sempre com referência simples.
- Split temporal: nunca embaralhar passado e futuro.
- Métricas: acompanhe MAE e RMSE no teste.
- Governança: documente dados, período e versão do modelo.
- Monitoramento: revise performance e recalibre quando necessário.
